This is evaluation for トキポナ1000文コーパス by ponaLM

PonaLM is still weak to evaluate correctly. So the fluency score might be wrong.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys, re, regex
sys.path.append("../../../ponaLM")
from models import PonaLM
import nltk
pd.set_option("display.max_colwidth", 1000)
pd.set_option('display.max_columns', 1000)

prepare toki pona language model and dictionary

In [2]:
with open('../../words.txt') as f:
    words = [word.rstrip() for word in f.readlines()]
    w2n = {w:i for i,w in enumerate(words)}

# for detect proper noun
def proper(word):
    return re.match(r'^([AIUEO]|[KSNPML][aiueo]|[TJ][aueo]|W[aie])n?(([ksnpml]?[aiueo]|[tj][aueo]|w[aie])n?)*$',word)

# for detect symbols
p_emoji = regex.compile(r'\p{Emoji=Yes}+')

def word_to_id(word):
    if word in w2n:
        return w2n[word]
    elif proper(word):
        return w2n['<Q>']
    elif word.isnumeric():
        return w2n['<NUM>']
    elif p_emoji.search(word):
        return w2n['<SYM>']
    else:
        assert mopu

In [3]:
model = PonaLM(len(words))
model.load_state_dict(torch.load('../../../ponaLM/lm.model', map_location='cpu'))
model.eval()

PonaLM(
  (embed): Embedding(145, 20)
  (lstmf): LSTM(20, 300)
  (lstmb): LSTM(20, 300)
  (h2h): Linear(in_features=300, out_features=20, bias=True)
  (h2o): Linear(in_features=20, out_features=145, bias=True)
  (drop1): Dropout(p=0.05)
  (drop2): Dropout(p=0.05)
  (drop3): Dropout(p=0.25)
  (drop4): Dropout(p=0.25)
)

code for evaluation

In [4]:
# Negotive Log Likelihhod
def NLL(sent):
    sent = [word_to_id(w) for w in sent]
    pred = model(torch.tensor([[0,0] + sent]), torch.tensor([[0,0] + sent[::-1]]))
    return [-p[w].item() for p,w in zip(pred[1:-1], sent)]

# evaluate the score of sentences
def evaluate(sent): 
    nlls = NLL(sent)
    prob = [round(np.exp(-l)*100,2) for l in nlls] # probability = exp(-NLL)
    fluency = round(1/(1+np.mean(nlls)),3)         # fluency = 1 / (1 + ave(all NLL))
    display(pd.DataFrame([prob + [fluency]], index=['prob'], columns=sent + ['Fluency']))

load corpus

In [5]:
sents = [sent.rstrip().split(' ') for sent in open('pona_corpus1000.txt')]
sents[0]

['mi', 'jan', 'Alito', '.']

evaluate fluency of each sentence

In [6]:
for i, line in enumerate(sents):
    print('#{:04}'.format(i+1))
    evaluate(line)

#0001


,mi,jan,Alito,.,Fluency
prob,1.39,43.64,53.48,87.86,0.406


#0002


,ali,li,ike,tawa,ona,.,Fluency
prob,1.22,83.6,6.81,50.08,7.54,88.26,0.36


#0003


...

,toki,Ikuwi,li,pona,ala,sama,toki,pona,.,Fluency
prob,2.42,33.61,93.55,4.48,15.11,0.02,3.89,15.7,78.14,0.274


#0119


,jan,sama,meli,lili,mi,li,pana,e,kasi,pi,suwi,lukin,tawa,mi,.,Fluency
prob,17.67,3.58,1.74,5.49,15.12,28.69,12.22,95.48,1.52,12.56,0.41,0.67,21.39,42.51,80.75,0.29


#0120


,kulupu,Nintento,li,sona,e,nasin,pali,pi,musi,.,Fluency
prob,0.09,5.91,95.46,4.83,67.8,1.47,1.44,10.88,1.05,82.5,0.258


some trial

In [7]:
def tokenize(sent):
    return nltk.tokenize.word_tokenize(sent)

In [8]:
evaluate(tokenize('moku mi li kili palisa.'))
evaluate(tokenize('moku mi li kili pi palisa.'))
evaluate(tokenize('moku mi li palisa kili.'))
evaluate(tokenize('moku mi li palisa pi kili.'))

,moku,mi,li,kili,palisa,.,Fluency
prob,0.05,7.38,10.81,0.02,0.04,82.18,0.173


,moku,mi,li,kili,pi,palisa,.,Fluency
prob,0.05,7.38,10.81,0.22,52.57,0.11,82.18,0.211


,moku,mi,li,palisa,kili,.,Fluency
prob,0.05,7.38,12.33,0.08,0.06,82.09,0.182


,moku,mi,li,palisa,pi,kili,.,Fluency
prob,0.05,7.38,12.33,0.14,61.83,0.18,82.09,0.213


In [9]:
evaluate(tokenize('nimi namako li nimi pu ala.'))
evaluate(tokenize('nimi namako li nimi pi pu ala.'))
evaluate(tokenize('nimi pi namako li nimi pi pu ala.'))

,nimi,namako,li,nimi,pu,ala,.,Fluency
prob,0.37,0.06,3.34,0.2,0.02,3.13,77.1,0.167


,nimi,namako,li,nimi,pi,pu,ala,.,Fluency
prob,0.37,0.06,3.34,0.39,32.27,0.01,3.13,77.1,0.182


,nimi,pi,namako,li,nimi,pi,pu,ala,.,Fluency
prob,1.24,16.19,0.04,3.34,0.39,32.27,0.01,3.13,77.1,0.196
